# Prueba de Kruskall-Wallis

### Objeto

Comparar 3 o más muestras para determinar si en el al menos en alguna existen diferencias significativas.
No indica en cual de la o las muestras existe tales diferencia. Se aplica Mann-Whitney por pares para determinar donde se encuentran la diferencia.

Restricciones:
* La distribución es **NO paramétrica**
* Los grupos son **independientes**
* Los datos son ordinales o coninuos

Ho: No hay diferencia significativa;

H1: Hay diferencia

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html">Scypy Kruskal</a>

## 0.Preparación librerias y definición de fórmulas# 

In [169]:
#Importacion
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import mannwhitneyu
from statsmodels.stats.diagnostic import normal_ad
from scipy.stats import kruskal

In [4]:
#Comprobación de Ho

def checkho(p_value_thresh, p_value):
    print("Para un p_value=",p_value)
    if p_value < p_value_thresh:
        print('Se rechaza la hipótesis nula, podemos asumir diferencias significativas')
    else:
        print('Se acepta la hipótesis nula, no podemos asumir diferencias significativas')

In [161]:
#Calculo factorial
def factorial(n):
    if n<=1:
        return 1
    return n*factorial(n-1) 



In [159]:
# Bonferroni:
def bonf (n):
    B =factorial(n)/(factorial(2)*factorial(n-2))
    return B

In [164]:
def compare_manwhitney(data,value, alpha,b):
    #Calculate alpha according to Bonferroni (b)
    alpha_new = alpha/b
    #To identify de unique values in the category
    test_list = data[value].unique()
    #To create all the possible couples
    res = [(a, b) for idx, a in enumerate(test_list) for b in test_list[idx + 1:]] 
    print('alpha corregida',alpha_new)
    #To calculate MannWhitney p-values from all the pairs
    for i in range(len(res)):
        group1=data[data[value]==res[i][0]]
        group2=data[data[value]==res[i][1]]
        # 'two-sided' alternative is choosen because we don´t know if the significance difference is bigger or smaller
        stat, p = mannwhitneyu(group1.rating, group2.rating,alternative='two-sided')
        print(res[i])
        print('      p-value:', p)
        if p > alpha_new:
            print('      No rechazamos H0: No hay diferencias significativas entre los grupos', res[i])
        else:
            print('      Rechazamos H0: Hay diferencias significativas entre los grupos', res[i])
        

## 1.Preanalisis de los datos

In [1]:
file = './Datos/drugstudy.csv'

In [4]:
# Carga de datos
df_data=pd.read_csv(file)

In [6]:
df_data.head()

,dose,rating
0,1.0,6.0
1,1.0,4.0
2,1.0,6.0
3,1.0,5.0
4,1.0,4.0


 ## 2.Comparando todo lso grupos con la prueba Kruskal-Wallis¶

Con la prueba de Kruskal-Wallis comprobamos si entre todos los grupos exisen diferencias significativas

In [157]:
num_categories = len(df_data.dose.unique())
num_categories

4

In [167]:
group1=df_data[df_data.dose==1.0]
group2=df_data[df_data.dose==2.0]
group3=df_data[df_data.dose==3.0]
group4=df_data[df_data.dose==4.0]

In [223]:
stat, p = kruskal (group1, group2, group3, group4)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# Interpretación
alpha = 0.05
if p > alpha:
	print('No rechazamos H0: No hay diferencias significativas entre los grupos.')
else:
	print('Rechazamos H0: Hay diferencias significativas entre los grupos.')

Statistics=298.284, p=0.000
Rechazamos H0: Hay diferencias significativas entre los grupos.


## 3.Comparación de pares de grupos con la prueba Mann-Whitney

Con Mann-Whitney a través de las comparaciones de pares vamos a encontrar donde hay diferencias significativas

In [165]:
b=bonf(num_categories)
b

6.0

In [163]:
compare_manwhitney(df_data, 'dose', 0.05,6)

alpha corregida 0.008333333333333333
(1.0, 2.0)
      p-value: 0.009911377465815045
      No rechazamos H0: No hay diferencias significativas entre los grupos (1.0, 2.0)
(1.0, 3.0)
      p-value: 0.00039457325940790667
      Rechazamos H0: Hay diferencias significativas entre los grupos (1.0, 3.0)
(1.0, 4.0)
      p-value: 0.0010607936166658369
      Rechazamos H0: Hay diferencias significativas entre los grupos (1.0, 4.0)
(2.0, 3.0)
      p-value: 0.07441228142144751
      No rechazamos H0: No hay diferencias significativas entre los grupos (2.0, 3.0)
(2.0, 4.0)
      p-value: 0.09993509549720665
      No rechazamos H0: No hay diferencias significativas entre los grupos (2.0, 4.0)
(3.0, 4.0)
      p-value: 1.0
      No rechazamos H0: No hay diferencias significativas entre los grupos (3.0, 4.0)
